In [ ]:
import torch
import os
from typing import List
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import albumentations as A
import numpy as np
import albumentations.pytorch as al_pytorch
from typing import Dict, Tuple
import pytorch_lightning as pl
torch.__version__

## Number of images

In [ ]:
# DATA_ROOT_PATH:str = "/kaggle/input/anime-sketch-colorization-pair/data/"
# NO_OF_IMAGES:int = 5 # Limit the number of images
# #
# train_path:str = f"{DATA_ROOT_PATH}train/"
# train_images:List[str] = [ f"{train_path}{x}" for x in os.listdir(train_path) ][:NO_OF_IMAGES]
# test_path:str = f"{DATA_ROOT_PATH}val/"
# test_images:List[str] = [ f"{test_path}{x}" for x in os.listdir(test_path) ][:NO_OF_IMAGES]
# #
# print(" Nb of train images : " + str(len(train_images)))
# print(" Nb of test images : " + str(len(test_images)))

## Let's plot a few images

In [ ]:
# plt.figure(figsize=(15,8))
# for i, img_path in enumerate(train_images[:9]):
#     im = Image.open(img_path)
#     plt.subplot(3,3,i+1)
#     plt.imshow(im)

# Create Model

In [ ]:
!git clone -b 16-create-v1-add-callback https://github.com/nmud19/thesisGAN.git

In [ ]:
os.chdir("thesisGAN")
from app.discriminator import patch_gan
from app.generator import unetGen
from app.consume_data import consume_data
from app.model import lit_model

# Set up gen and discriminator

In [ ]:
lr = 0.0002
generator = unetGen.UNET()
discriminator = patch_gan.PatchGan(
    input_channels=6,
    hidden_channels=1
)

# Setup the data

In [ ]:
anime_sketch_data_module = consume_data.AnimeSketchDataModule(
    data_dir="/kaggle/input/anime-sketch-colorization-pair/data/",
    train_num_images=0,
    val_num_images=1000,
    train_batch_size=8,
    val_batch_size=1,
)

# Add the pytorch lightning script

In [ ]:
# Trainer
from app.model.lit_model import Pix2PixLitModule

model = Pix2PixLitModule(
    generator=generator,
    discriminator=discriminator,
    use_gpu=False
)
# Trainer
# epoch_inference_callback = lit_model.EpochInference(valid_dataloader, use_gpu=False)
# checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint()
logger = pl.loggers.TensorBoardLogger("tb_logs", name="pix2pix_lightning_model")
trainer = pl.Trainer(
    #fast_dev_run=True,
    max_epochs=100,
    logger=logger,
#     callbacks=[
#         epoch_inference_callback,
#         checkpoint_callback,
#         pl.callbacks.TQDMProgressBar(refresh_rate=10)
#     ],
    default_root_dir="chk",
    tpu_cores=8,
    progress_bar_refresh_rate=20,
    #accelerator='gpu', 
    #devices=1
#     gpus=-1
)
trainer.fit(
    model=model,
    datamodule=anime_sketch_data_module
)

In [ ]:
!ls 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs